# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame
*For part II a dataframe is created
*that contains the following:
*✓ Ten or less rows.
*✓ City
*✓ Country
*✓ Latitude
*✓ Longitude
*✓ Hotel Name

In [11]:
transposed_data = pd.read_csv("/Users/alexispalmer/Desktop/Github2/Python-API-Challenge/WeatherPy/output_data/what_weatherpy_data.csv")
transposed_data.head(10)

,Unnamed: 0,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
0,0,hobart,AU,75,81,1587333184,51.80,6.93,-42.88,147.33
1,1,rikitea,PF,100,68,1587333375,76.87,6.96,-23.12,-134.97
2,2,ambulu,ID,100,76,1587333225,78.46,2.91,-8.35,113.61
3,3,igis,CH,100,60,1587333227,54.59,3.33,46.95,9.57
4,4,cape town,ZA,75,77,1587333059,64.00,17.22,-33.93,18.42
5,5,dikson,RU,95,89,1587333231,34.93,29.04,73.51,80.55
6,6,ellensburg,US,1,34,1587333233,66.20,2.91,47.00,-120.55
7,7,belushya guba,ZA,73,96,1587332969,60.01,1.99,-34.42,19.23
8,8,hermanus,US,1,34,1587333149,51.80,23.04,48.11,-98.87
9,9,devils lake,AR,75,81,1587333140,42.80,23.04,-54.80,-68.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Configure gmaps with my google api key from the config file
gmaps.configure(api_key='g_key')

In [13]:
#Covert the lat and lngs to location(int to float) & Humidity as the weight(int to float)
locations = transposed_data[['Latitude', 'Longitude']].astype(float)
weight = transposed_data['Humidity'].astype(float)

In [14]:
 # Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Make sure to drop any rows that don't contain all three conditions. (Longtitude, Latitude, Max Temp(f), WindSpeed(mph), Cloudiness( %)
* A max temperature lower than 80 degrees but higher than 70. #weather_df = weather_df (max_temp) <80 & weather_df [max temp ] > 70
* Wind speed less than 10 mph. #weather_df = weather_df[wind speed] <10
* Zero cloudiness. #no_cloud = weather_df[ weather [cloudiness (%)] == 0

In [15]:
#Create a New Datafame with conditons 
best_weather_df = transposed_data
#Keep these three columns/conditions =  Max Temp(f), WindSpeed(mph), Cloudiness( %)

#A max temperature lower than 80 degrees but higher than 70.
best_weather_df = best_weather_df.loc[(best_weather_df['Max Temp (F)'] <80) & (best_weather_df['Max Temp (F)'] > 70)]
best_weather_df

#Wind speed less than 10 mph. 
best_weather_df = best_weather_df.loc[best_weather_df['Wind Speed (mph)'] <10]
best_weather_df

#Zero cloudiness 
best_weather_df = best_weather_df.loc[best_weather_df['Cloudiness %'] == 0]
best_weather_df

# Drop any rows will null values.
best_weather_df = best_weather_df.dropna()
best_weather_df

,Unnamed: 0,City,Country,Cloudiness %,Humidity,Date,Max Temp (F),Wind Speed (mph),Latitude,Longitude
232,232,grande-riviere,AR,0,39,1587333596,79.93,3.87,-26.79,-60.44
358,358,lovozero,EG,0,22,1587333779,72.09,5.41,29.20,25.52
367,367,raga,BR,0,63,1587333789,70.92,7.49,-32.57,-53.38
475,475,marcona,MR,0,17,1587333911,75.74,9.08,20.52,-13.05
497,497,mackay,TH,0,86,1587333935,77.11,1.19,12.50,102.50
501,501,porto velho,AR,0,61,1587333825,72.00,4.00,-34.65,-58.62
525,525,sobolevo,BR,0,51,1587333967,71.80,3.56,-29.68,-53.81
564,564,kawana waters,BR,0,45,1587334011,74.75,3.36,-30.39,-56.45
572,572,komsomolskiy,VN,0,81,1587334020,78.53,3.69,9.18,105.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Store variable named hotel_df
hotel_df = best_weather_df

# geocoordinates - 06-Python-APIs/3/Activities/02-Ins_Google_Places/Solved/Google_Places.ipynb
target_coordinates = ["-26.79, -60.44", "29.20, 25.52", "-32.57, -53.38", "20.52, -13.05", "12.50, 102.50", 
                      "-34.65, -58.62","-29.68, -53.81", "-30.39, -56.45", "9.18, 105.15"]
target_radius = 5000
target_type = "hotels"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
hotel_response = requests.get(base_url, params=params).json()
# Print the json (pretty printed)
print(json.dumps(hotel_response, indent=4, sort_keys=True))

{
    "error_message": "This API project is not authorized to use this API.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map